# 创建连接

任何对话助手，都是先从创建一个模型服务连接开始。
下面是未采用任何框架的连接方式：

In [ ]:
from openai import OpenAI

# 以本地ollama服务为例
client = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
# 使用模型提供商
# client = OpenAI(api_key = "你的api_key", base_url = "提供商的服务地址")

response = client.chat.completions.create(
    model = 'qwen3', 
    messages = [
        {"role": "system", "content": "你是一名诗歌创作高手，尤其擅长中国古代诗歌。"},
        {"role": "user", "content": "请创作一首跟端午有关的七言绝句。"},
    ])
# print(response)
print(response.choices[0].message.content)

下面是采用LangChain的连接方式：

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_ollama.llms import OllamaLLM

llm = ChatOpenAI(
        api_key = 'ollama',
        base_url = 'http://localhost:11434/v1',
        model = 'qwen3'
        # other params...
    )

#或者

llm = OllamaLLM(
        base_url = 'http://localhost:11434', 
        model = 'qwen3')

基于 `ChatOllama` 或 `init_chat_model` 的实现

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage

llm = ChatOllama(model = "qwen3",
    base_url = 'http://localhost:11434', 
    temperature = 0.8,
    num_predict = 256)
# model = init_chat_model("gpt-4", model_provider="openai")

messages = [
        SystemMessage(content="你是一个乐于助人的助手，能做中英文互译。翻译用户句子。"),
        HumanMessage(content="我想成为一位超人！")]

# 第一种调用
#response = llm.invoke(messages)
#print(response)

# 异步调用
# await llm.ainvoke(messages)

# 第二种，流式调用
#for chunk in llm.stream(messages):
#    print(chunk.text(), end="")

# 异步调用
async for chunk in llm.astream(messages):
    print(chunk.content)

如果想查看演示效果，可以使用Gradio。

In [ ]:
import gradio as gr

def chat(message, history):
    client = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

    response = client.chat.completions.create(
        model = 'qwen3', 
        messages = [
            {"role": "system", "content": "你是一名诗歌创作高手，尤其擅长中国古代诗歌。"},
            {"role": "user", "content": "请创作一首跟端午有关的七言绝句。"},
        ])
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()
# 或者
#interface = gr.Interface(fn=print_text, inputs="text", outputs="text")
#interface.launch(share=True, server_port=7860)

# 流失输出

In [ ]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(
    base_url = 'http://localhost:11434', 
    model = 'qwen3'
)

for chunk in llm.stream("海水为什么是蓝的？"):
    print(chunk, end="|", flush=True)

# 基于模板

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

llm = OllamaLLM(
    base_url = 'http://localhost:11434', 
    model = 'qwen3'
)

template = """Question: {question}
    Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm
response = chain.invoke({"question": "海水为什么是蓝的？"})
print(response)

# chain

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = OllamaLLM(
    base_url = 'http://localhost:11434', 
    model = 'qwen3'
)

prompt = ChatPromptTemplate.from_messages([
        ("system", "You are world class technical documentation writer."),
        ("user", "{input}")
    ])

output_parser = StrOutputParser()

chain = prompt | llm | output_parser
question = "how can langsmith help with testing?"
response = chain.invoke({"input": question})
print(response)